# KNN dan Decision Tree (UTS)


## K-NN Algorithm 


1.   Baca data bunga Breast Cancer Coimbra
2.   Implementasi algoritma K-NN untuk mencari kelas dari data tes



## Baca Data 



*   Baca data dengan *library pandas* *italicized text*


#### Persiapan Lingkungan


*   Impor *train_test_split* dari *sklearn*
*   Impor *KNeighborsClassifier* dari *sklearn*
*   Impor *pandas library*



In [1]:
from sklearn.model_selection import train_test_split #split dataset into train and test data

In [2]:
from sklearn.neighbors import KNeighborsClassifier# Create KNN classifier

In [3]:
import pandas as pd

In [4]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)

In [5]:
dataset_url = "https://raw.githubusercontent.com/akhmadamanulloh/main/main/dataR2.csv"

In [6]:
df = pd.read_csv(dataset_url)
df.head()

,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification
0,48,23.500000,70,2.707,0.467409,8.8071,9.702400,7.99585,417.114,1
1,83,20.690495,92,3.115,0.706897,8.8438,5.429285,4.06405,468.786,1
2,82,23.124670,91,4.498,1.009651,17.9393,22.432040,9.27715,554.697,1
3,68,21.367521,77,3.226,0.612725,9.8827,7.169560,12.76600,928.220,1
4,86,21.111111,92,3.549,0.805386,6.6994,4.819240,10.57635,773.920,1


In [7]:
#check number of rows and columns in dataset
df.shape

(116, 10)

### Hitung Data

*   Pisahkan kolom "variety" dari *dataframe*
*   Ambil kolom "variety" sebagai target kolom untuk kategori kelas
*   Pisahkan data latih dengan data tes


```
# data latih (nilai data)
X_train 

# data tes (nilai data)
X_test 

# data latih (kelas data)
y_train

# data tes (kelas data)
y_test
```





In [8]:
#create a dataframe with all training data except the target column
X = df.drop(columns=["Classification"])#check that the target variable has been removed
X.head()

,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1
0,48,23.500000,70,2.707,0.467409,8.8071,9.702400,7.99585,417.114
1,83,20.690495,92,3.115,0.706897,8.8438,5.429285,4.06405,468.786
2,82,23.124670,91,4.498,1.009651,17.9393,22.432040,9.27715,554.697
3,68,21.367521,77,3.226,0.612725,9.8827,7.169560,12.76600,928.220
4,86,21.111111,92,3.549,0.805386,6.6994,4.819240,10.57635,773.920


In [9]:
# separate target values
y = df["Classification"].values  # view target values
y[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [10]:
# percent_amount_of_test_data = / HUNDRED_PERCENT
percent_amount_of_test_data = 0.2

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = percent_amount_of_test_data, random_state=1, stratify=y)



*   Tentukan nilai K = 10 (10 tetangga terdekat)
*   Buat data latih


```
knn_classifier.fit(train_data)
```




In [12]:
amount_of_neighbor = 10

In [13]:
# Fit the classifier to the data
knn = KNeighborsClassifier(n_neighbors = amount_of_neighbor)

# create train data
knn.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=10)



*   Tes data target terhadap data latih
*   Tampilkan predksi 10 data pertama



```
knn_classifier.predict(test_data)
```





In [14]:
#show first 5 model predictions on the test data
knn.predict(X_test)

array([1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
       1, 2])



*   Cetak prediksi nilai dari data tes



```
knn_classifier.score(test_data)
```





In [15]:
#check accuracy of our model on the test data
knn.score(X_test, y_test)

0.5

*  Cek sebuah data dengan nilai 


```
# Test Data (1)
[
    [
        45, 
        21.30394858, 
        102, 
        13.852, 
        3.4851632, 
        7.6476, 
        21.056625, 
        23.03408,	
        552.444
    ]
]
```



In [16]:
# Custom value to predict (1) based on manual calculation
knn.predict([[45, 21.30394858, 102, 13.852, 3.4851632, 7.6476, 21.056625, 23.03408,	552.444]])

array([1])

In [17]:
# Custom value to predict (2)
knn.predict([[72,25.59,82,2.82,0.570392,24.96,33.75,3.27,392.46]])

array([2])

In [18]:
# Custom value to predict (3) based on manual calculation
knn.predict([[20,35,   79,2.5, 0.4,     25,   32,   3.1, 393]])

array([1])

### Kesimpulan


*   Dataset Breast Cancer Combria terdiri dari 116 data
*   Ambil **80%** dari data Breast Cancer Combria sebagai **data latih**
*   Ambil **20%** dari data Breast Cancer Combria sebagai **data tes**
*   Tetapkan nilai K = 10
*   Hasil 10 data pertama dari 20% data latih


```
"""
1 = Healthy controls 
2 = Patients
"""
[1, 1, 1, 2, 1, 2, 2, 2, 2, 1]
```
*   Dengan akurasi data yaitu $ 0.5\% $





## Implementasi Decision Tree
